In [39]:
VERBOSE = str(os.getenv("YOLO_VERBOSE", True)).lower() == "False"  # global verbose mode

In [1]:
%pip install ultralytics

^C
Note: you may need to restart the kernel to use updated packages.


In [40]:
import ultralytics
ultralytics.__version__

'8.3.21'

In [80]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import os


os.environ["YOLO_VERBOSE"] = "false"


model=YOLO('yolov5n.pt')

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [81]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [88]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [89]:
#llegir el video
cap=cv2.VideoCapture('./output7.mp4')

In [90]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]

In [91]:
import time
import numpy as np

# Initialize the FPS counter
fps = 0
start_time = time.time()
frame_difference = np.array([])

pts = np.array([[359, 340],[186, 371] ,[8, 953], [446, 951]], np.int32)
# divide all values by 2
pts = pts // 2
pts = pts.reshape((-1, 1, 2))

last_frame = None

while True:
    skip = False
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    # RESOLUCIO IMATGE ORIGINAL: 960X540
    # BAIXAR RESOLUCIO
    frame = cv2.resize(frame, (270, 480))
    original_frame = frame.copy()

    mask = np.zeros_like(frame, dtype=np.uint8)
    cv2.fillPoly(mask, [pts], (255, 255, 255))

    if last_frame is not None:
        frame_difference = cv2.absdiff(last_frame, frame)*2


        frame_difference = cv2.bitwise_and(frame_difference, mask)

        print(frame_difference.mean())

        cv2.imshow("frame_difference", frame_difference)





    

    # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
    if count % 2 != 0 or frame_difference.mean() < 1:
        fps += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            current_fps = fps / elapsed_time
            cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # red line
            cv2.line(frame, (0, 450), (850, 450), (255, 0, 0), 3)  # blue line
            cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            

    # Convertir el frame a grisos per anar més ràpid: 
    # No es pot utilitzar perquè YOLO treballa en RGB
    # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    else:
        # YOLO Prediction
        results = model.predict(frame)

        # aconseguint les dades de la bounding box i confiança
        a = results[0].boxes.data
        a = a.detach().cpu().numpy()
        px = pd.DataFrame(a).astype("float")

        # only work with car class. only count cars. Quedarnos bounding box dels cars
        car_bboxes = []
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])
            c = class_list[d]
            if 'car' in c:
                car_bboxes.append([x1, y1, x2, y2])

        # provide les coordinades al tracker
        bbox_id = tracker.update(car_bboxes)

        # draw bbox
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int((x3 + x4) // 2)
            cy = int((y3 + y4) // 2)

            # Draw bounding box and ID
            # cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
            # cv2.putText(frame, f'ID: {id}', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Update down/up counters
            red_line_y = 400
            blue_line_y = 450
            offset = 7

            # DOWNWARDS condition: red line first, then blue line
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                down[id] = cy
            if id in down:
                if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # red dot
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_down.append(id)  # list

            # UPWARDS condition: blue line first, then red line
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                up[id] = cy
            if id in up:
                if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # red dot
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_up.append(id)  # list

        # Draw lines
        cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # red line
        cv2.line(frame, (0, 450), (850, 450), (255, 0, 0), 3)  # blue line

        # Show counters on the frame
        downwards = len(set(counter_down))
        upwards = len(set(counter_up))



        # FPS calculation
        fps += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            current_fps = fps / elapsed_time
            cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        
    cv2.imshow("frames", frame)
    # record to video

    
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
    last_frame = original_frame

cap.release()
cv2.destroyAllWindows()


0.44424897119341566
0.3826954732510288
0.28337962962962965
0.24583847736625514
0.2361522633744856
0.2679218106995885
0.289022633744856
0.24428497942386831
0.3048045267489712
0.28507201646090535
0.3911985596707819
0.8882201646090535
0.2733333333333333
0.30915637860082307
0.2347633744855967
0.22770576131687242
0.25823045267489714
0.2290946502057613
0.2092849794238683
0.21886316872427983
0.1790432098765432
0.35158436213991767
0.26709876543209876
0.8256995884773662
0.21422325102880657
0.2622119341563786
0.2327469135802469
0.3165380658436214
0.24583333333333332
0.26053497942386833
0.28756687242798357
0.2252366255144033
0.20646604938271604
0.2124897119341564
0.22030349794238682
0.8200668724279835
0.290298353909465
0.2559927983539095
0.23123456790123456
0.2594701646090535
0.21420267489711933
0.22106995884773661
0.21555555555555556
0.19037551440329217
0.24781378600823045
0.23295267489711935
0.2474537037037037
0.7874434156378601
0.275570987654321
0.24934670781893004
0.274809670781893
0.35394032

In [9]:
cap.release()
cv2.destroyAllWindows()